In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

C:\Users\donov\AppData\Local\Temp\ipykernel_21044\2479383241.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('C:/Users/donov/Documents/train.csv')

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Feature Engineering
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.isocalendar().week
data['DayOfWeek'] = data['Date'].dt.dayofweek

# Lag features
data['Lag_1'] = data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
data['Lag_2'] = data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)
data['Lag_3'] = data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(3)
data.dropna(inplace=True)

C:\Users\donov\AppData\Local\Temp\ipykernel_21044\414333532.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [3]:
sarima_model = SARIMAX(data['Weekly_Sales'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_results = sarima_model.fit()
sarima_forecast = sarima_results.forecast(steps=30)

c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an excepti

In [4]:
X = data[['Store', 'Dept', 'Year', 'Month', 'Week', 'DayOfWeek', 'Lag_1', 'Lag_2', 'Lag_3']]
y = data['Weekly_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

print('Random Forest MAE:', mean_absolute_error(y_test, rf_predictions))
print('Random Forest RMSE:', mean_squared_error(y_test, rf_predictions, squared=False))

Random Forest MAE: 1639.4344838663053
Random Forest RMSE: 4810.448675556847


c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [5]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

print('XGBoost MAE:', mean_absolute_error(y_test, xgb_predictions))
print('XGBoost RMSE:', mean_squared_error(y_test, xgb_predictions, squared=False))

XGBoost MAE: 1596.1979457838268
XGBoost RMSE: 4071.4051722425056


c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
# Reshape data for LSTM
X_lstm = np.array(data[['Year', 'Month', 'Week', 'DayOfWeek', 'Lag_1', 'Lag_2', 'Lag_3']], dtype=np.float32)
y_lstm = np.array(data['Weekly_Sales'], dtype=np.float32)
X_lstm = X_lstm.reshape((X_lstm.shape[0], 1, X_lstm.shape[1]))

X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, X_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

# Train LSTM model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=32, verbose=0)
lstm_predictions = lstm_model.predict(X_test_lstm)

print('LSTM MAE:', mean_absolute_error(y_test_lstm, lstm_predictions))
print('LSTM RMSE:', mean_squared_error(y_test_lstm, lstm_predictions, squared=False))

c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2574/2574 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
LSTM MAE: 1952.6268
LSTM RMSE: 6024.367


c:\Users\donov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
